## Задание 4

1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news
Для парсинга использовать xpath. Структура данных должна содержать:
название источника,
наименование новости,
ссылку на новость,
дата публикации

2)Сложить все новости в БД

In [56]:
from lxml import html
from requests import get
from pprint import pprint
from pymongo import MongoClient
import re

In [2]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}

### Обработка новостей lenta.ru

In [36]:
response_lenta = get('https://lenta.ru/', headers = header)
root = html.fromstring(response_lenta.text)
result = []
items = root.xpath(".//div[@class='span4']/div[@class='item'] | .//div[@class='first-item']")
print(*items)

<Element div at 0x56b9990> <Element div at 0x56b9960> <Element div at 0x6556660> <Element div at 0x6556690> <Element div at 0x658b120> <Element div at 0x658bb10> <Element div at 0x658b9f0> <Element div at 0x658b780> <Element div at 0x658b870> <Element div at 0x658b9c0>


In [37]:
for item in items:
    lenta_info = {}
    date = item.xpath(".//time[@class='g-time']//@title")
    link1 = item.xpath(".//a/@href")
    link = f'https://lenta.ru{link1[0]}'
    name = item.xpath(".//a//text()")    
     
    lenta_info['link'] = link
    lenta_info['name'] = name[1]
    lenta_info['date'] = date
    lenta_info['source'] = 'lenta.ru'
   
    result.append(lenta_info)


### Обработка новостей yandex.news

In [78]:
response_ya = get('https://yandex.com/news/', headers = header)
root = html.fromstring(response_ya.text)
items = root.xpath(".//div[@role='main']//div[position() = 4]//td[@class='stories-set__item'] | .//div[@role='main']//div[position() = 5]//td[@class='stories-set__item']")
print(*items)

<Element td at 0x65724b0> <Element td at 0x6572870> <Element td at 0x6572360> <Element td at 0x65725a0> <Element td at 0x65721b0> <Element td at 0x6572d20> <Element td at 0x6a3a570> <Element td at 0x6a3a450> <Element td at 0x6a3a7b0> <Element td at 0x6a3adb0>


In [76]:
for item in items:
    ya_info = {}
    date1 = item.xpath(".//div[@class='story__date']//text()")
    date = 'Сегодня '+date1[0].split()[-1]
    source = re.sub(r'[\d\W]', '', str(date1))
    link1 = item.xpath(".//a/@href")
    link = f'https://yandex.com{link1[0]}'         
    name = item.xpath(".//a//text()")    
    
    ya_info['link'] = link
    ya_info['name'] = name
    ya_info['date'] = date
    ya_info['source'] = source
    
    result.append(ya_info)
   

### Обработка новостей news.mail.ru

In [105]:
response_mail = get('https://news.mail.ru/', headers = header)
root = html.fromstring(response_mail.text)
items = root.xpath(".//div[contains(@class,'daynews__item')]/@href | .//ul[@class='list list_type_square list_half js-module']/@href")
print(items)

[]


In [104]:
for item in items:
    link = root.xpath("./@href")
    print(link)
    mail_info = {}
    response_item = get(f'https://news.mail.ru{link}', headers = header)
    root_item = html.fromstring(response_item.text)
    root2 = root_item.xpath(".//div[@class='article js-article js-module']")
    print(type(root_item))
    name = root2.xpath(".//h1//text()")
    print(response_item)
    
    #mail_info['link'] = f'https://news.mail.ru{link}'
    #mail_info['date'] = date
    #mail_info['source'] = source    
    #mail_info['name'] = name


[]


ConnectionError: HTTPSConnectionPool(host='news.mail.ru[]', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x065F90F0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

### Сложить все новости в БД

In [106]:
client = MongoClient('localhost', 27017)

In [107]:
db = client['news_database']

In [108]:
news = db.news

In [109]:
news.insert_many(result)